In [13]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('/content/drive/My Drive/Colab Notebooks/pizzabot/intents.json').read()


In [9]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


In [10]:
data = next(iter(uploaded.values()))
type(data)

bytes

In [11]:
intents=json.loads(data.decode())

In [15]:
nltk.download('punkt')
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Hey', 'Pbot'], 'Pbot'), (['Hii', 'Pbot'], 'Pbot'), (['Pbot'], 'Pbot'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['pizza'], 'pizza'), (['Can', 'you', 'book', 'a', 'Pizza', '?'], 'pizza'), (['Order', 'Pizza', '!', '!'], 'pizza'), (['what', 'are', 'my', 'options'], 'types'), (['types', 'available'], 'types'), (['My', 'available', 'choices'], 'types

In [17]:
nltk.download('wordnet')
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
40 documents
9 classes ['Pbot', 'Size', 'count', 'goodbye', 'greeting', 'options', 'pizza', 'thanks', 'types']
61 unique lemmatized words ["'s", '1', '2', '3', '4', '5', '6', 'a', 'add', 'anyone', 'are', 'available', 'awesome', 'book', 'bye', 'can', 'chatting', 'cheese', 'choice', 'day', 'extra', 'for', 'good', 'goodbye', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hii', 'hola', 'how', 'i', 'is', 'later', 'me', 'more', 'my', 'next', 'nice', 'of', 'option', 'order', 'pbot', 'peri', 'pizza', 'quantity', 'see', 'size', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'topping', 'type', 'want', 'what', 'you']


In [18]:

training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


In [19]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

Epoch 1/200
8/8 [==============================] - 0s 1ms/step - loss: 2.2107 - accuracy: 0.1500
Epoch 2/200
8/8 [==============================] - 0s 1ms/step - loss: 2.1949 - accuracy: 0.2000
Epoch 3/200
8/8 [==============================] - 0s 1ms/step - loss: 2.0846 - accuracy: 0.3500
Epoch 4/200
8/8 [==============================] - 0s 1ms/step - loss: 2.1178 - accuracy: 0.1750
Epoch 5/200
8/8 [==============================] - 0s 1ms/step - loss: 2.0272 - accuracy: 0.3500
Epoch 6/200
8/8 [==============================] - 0s 1ms/step - loss: 1.9864 - accuracy: 0.3000
Epoch 7/200
8/8 [==============================] - 0s 1ms/step - loss: 1.9431 - accuracy: 0.3250
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 1.7828 - accuracy: 0.4000
Epoch 9/200
8/8 [==============================] - 0s 1ms/step - loss: 1.7419 - accuracy: 0.4250
Epoch 10/200
8/8 [==============================] - 0s 1ms/step - loss: 1.7315 - accuracy: 0.5000
Epoch 11/200
8/8 [===========

In [21]:
model.save('my_model.h5') 